In [1]:
from Player import Player
from Trainer import Trainer
from TetrisModel import TetrisModel
from Pretrainer import Pretrainer
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import time

In [3]:
tf.range(9, -1, -1)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])>

In [2]:
piece_dim = 8
key_dim = 12
depth = 16
gamma = 0.99
lam = 0.95

In [19]:
# Use lambda instead of gamma to immitate shape of gae without value predictions
pretrainer = Pretrainer(gamma=lam)

In [20]:
players_data = pretrainer._load_data()

In [21]:
pretrainer._load_dset(players_data)

1.00

In [3]:
# max_len = pretrainer._max_len
max_len = 7

In [23]:
gt_dset = pretrainer._cache_dset()

Done Caching


In [4]:
agent = TetrisModel(piece_dim=piece_dim,
                    key_dim=key_dim,
                    depth=depth,
                    num_heads=4,
                    num_layers=4,
                    max_length=max_len)

In [5]:
logits, values = agent((tf.random.uniform((1, 28, 10, 1)),
                        tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                        tf.random.uniform((1, max_len-1), minval=0, maxval=key_dim, dtype=tf.int32)))
agent.summary(), tf.shape(logits), tf.shape(values)

Model: "tetris_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (1, 70, 16)               4800      
                                                                 
 seq_embedding (SeqEmbedding  multiple                 128       
 )                                                               
                                                                 
 seq_embedding_1 (SeqEmbeddi  multiple                 192       
 ng)                                                             
                                                                 
 pdec_0 (DecoderLayer)       multiple                  9776      
                                                                 
 pdec_1 (DecoderLayer)       multiple                  9776      
                                                                 
 pdec_2 (DecoderLayer)       multiple                 

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1,  6, 12])>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 6, 1])>)

In [6]:
agent_optimizer = keras.optimizers.Adam(learning_rate=1e-6)
agent.compile(optimizer=agent_optimizer)

In [7]:
ref_agent = TetrisModel(piece_dim=piece_dim,
                        key_dim=key_dim,
                        depth=depth,
                        num_heads=4,
                        num_layers=4,
                        max_length=max_len)

In [8]:
logits, values = ref_agent((tf.random.uniform((1, 28, 10, 1)),
                            tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                            tf.random.uniform((1, max_len-1), minval=0, maxval=key_dim, dtype=tf.int32)))

In [9]:
agent.load_weights('agent_weights_small/agent_finetuned')

In [10]:
ref_agent.load_weights('agent_weights_small/agent_reference')

In [11]:
epochs = 10

In [ ]:
losses, accs = pretrainer.train(agent, gt_dset, epochs)

In [ ]:
plt.plot(losses)
plt.plot(accs)

In [19]:
if 'y' in input('YOU SURE?????'):
    ref_agent.save_weights('agent_weights_small/agent_reference')

YOU SURE????? y


In [11]:
%matplotlib qt

In [12]:
optimizers = keras.optimizers.Adam(learning_rate=1e-6), keras.optimizers.Adam(learning_rate=1e-6)

In [13]:
trainer = Trainer(model=agent,
                  ref_model=ref_agent,
                  optimizers=optimizers,
                  max_len=max_len,
                  gamma=gamma,
                  lam=lam,
                  max_episode_steps=100,
                  buffer_cap=1000)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michaelsherrick. Use `wandb login --relogin` to force relogin


In [14]:
trainer.fill_replay_buffer()

Done filling replay buffer


In [15]:
trainer.train(gens=10000, train_steps=10, training_actor=True)

Current Gen: 1	|	Avg Reward: 0.0	|	Total Reward: 1.9	|
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Current Gen: 2	|	Avg Reward: 0.0	|	Total Reward: 6.7	|97	|	
Current Gen: 3	|	Avg Reward: 0.0	|	Total Reward: 3.0	|36	|	
Current Gen: 4	|	Avg Reward: 0.0	|	Total Reward: 3.8	|69	|	
Current Gen: 5	|	Avg Reward: 0.0	|	Total Reward: 0.8	|39	|	
Current Gen: 6	|	Avg Reward: 0.0	|	Total Reward: 0.5	|02	|	
Current Gen: 7	|	Avg Reward: 0.0	|	Total Reward: 2.6	|95	|	
Current Gen: 8	|	Avg Reward: 0.0	|	Total Reward: 3.5	|77	|	
Current Gen: 9	|	Avg Reward: 0.0	|	Total Reward: 1.0	|83	|	
Current Gen: 10	|	Avg Reward: 0.0	|	Total Reward: 0.4	|3	|	
Current Gen: 11	|	Avg Reward: 0.0	|	Total Reward: 1.3	|0	|	
Current Gen: 12	|	Avg Reward: 0.0	|	Total Reward: 2.5	|8	|	
Current Gen: 13	|	Avg Reward: 0.0	|	Total Reward: 2.1	|6	|	
Current Gen: 14	|	Avg Reward: 0.0	|	Total Reward: 1.3	|5	|	
Current Gen: 15	|	Avg Reward: 0.0	|	Total Reward: 3.9	|1	|	
Current Gen: 16


KeyboardInterrupt



In [16]:
episode_data = trainer.player.run_episode(agent, max_steps=100, greedy=True, renderer=trainer.renderer)

In [17]:
episode_boards, episode_pieces, episode_inputs, episode_actions, episode_valid, episode_probs, episode_values, episode_rewards = episode_data

In [18]:
episode_advantages, episode_returns = trainer._compute_gae(episode_values, episode_rewards, trainer.gamma, trainer.lam)

In [19]:
fig, ax = plt.subplots()
ax.plot(episode_returns, label='Returns')
ax.plot(episode_rewards, label='Rewards')
ax.plot(episode_values, label='Values')
ax.plot(episode_advantages, label='Advantages')
ax.legend()
tf.reduce_sum(episode_rewards)

<tf.Tensor: shape=(), dtype=float32, numpy=5.75>